<a href="https://colab.research.google.com/github/iakovkh/mypython/blob/main/LSTM_example30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Configuration section
data_file_name = '/content/sample_data/NN_cut.xlsx'
validate_file_name = '/content/sample_data/NN_control_0.xlsx'
output_file_name = 'output.xlsx'
actual_file_name = '/content/sample_data/NN_control.xlsx'
normalize_data = True
train_ratio = 0.8
optimizer_name = 'Adam'  # Other options: 'SGD', 'RMSprop', etc.
loss_function_name = 'MSELoss'  # Other options: 'CrossEntropyLoss', 'NLLLoss', etc.
activation_function_name = 'ReLU'  # Other options: 'Sigmoid', 'Tanh', etc.
epochs = 500
network_structure = [(1, 5), (2, 3), (3, 2)]
conv_filter_size = 3

# Step I - General data preparation
class MyMatrix:
    def __init__(self):
        self.data = None

    def read_data_from_file(self, file_name):
        self.data = pd.read_excel(file_name)

    def normalize_data(self):
        scaler = MinMaxScaler()
        self.data.iloc[:, 1:] = scaler.fit_transform(self.data.iloc[:, 1:])

    def display_console(self, column=None):
        if column:
            print(self.data.iloc[:, column])
        else:
            print(self.data)

    def display_visual(self, column=None):
        if column:
            plt.plot(self.data.iloc[:, column])
            plt.show()
        else:
            self.data.iloc[:, 1:].plot(subplots=True)
            plt.show()

# Step II – Read and prepare data
MyData = MyMatrix()
MyData.read_data_from_file(data_file_name)

# Split data into training and testing sets
train_data, test_data = train_test_split(MyData.data, train_size=train_ratio, random_state=42)

if normalize_data:
    MyData.normalize_data()

# Step III – Design the neural network
class ConvNet(nn.Module):
    def __init__(self, input_size, network_structure, activation_function):
        super(ConvNet, self).__init__()
        layers = []
        in_channels = 1
        for layer_num, out_channels in network_structure:
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=conv_filter_size))
            layers.append(activation_function())
            layers.append(nn.MaxPool2d(kernel_size=2))
            in_channels = out_channels
        self.conv = nn.Sequential(*layers)
        self.fc = nn.Linear(out_channels, 1)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Prepare data for the neural network
X_train = torch.tensor(train_data.iloc[:, 1:].values, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(train_data.iloc[:, 0].values, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(test_data.iloc[:, 1:].values, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(test_data.iloc[:, 0].values, dtype=torch.float32).unsqueeze(1)

# Initialize the neural network
activation_function = getattr(nn, activation_function_name)
model = ConvNet(X_train.shape[1], network_structure, activation_function)

# Define the loss function and optimizer
loss_function = getattr(nn, loss_function_name)()
optimizer = getattr(optim, optimizer_name)(model.parameters())

# Train the neural network
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = loss_function(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Step IV - Testing results
model.eval()
with torch.no_grad():
    predictions = model(X_test).numpy()

# Save the results to an Excel file
output_df = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.numpy().flatten()})
output_df.to_excel(output_file_name, index=False)

# Visualize the results
output_df.plot()
plt.show()

# Calculate and print the correlation
correlation = np.corrcoef(output_df['Predicted'], output_df['Actual'])[0, 1]
print(f'Correlation between predicted and actual values: {correlation:.4f}')


RuntimeError: Given groups=1, weight of size [5, 1, 3, 3], expected input[1, 1999, 1, 32] to have 1 channels, but got 1999 channels instead